# NLP基礎 (補足): Hugging Face Tokenizers体験

これまでのノートブックで、サブワード分割の重要性と、BPE、WordPiece、SentencePieceといった代表的なアルゴリズムの基本的な考え方について学びました。
実際の研究や開発では、これらのアルゴリズムを効率的に利用するために、Hugging Face `transformers` ライブラリや `tokenizers` ライブラリのような高機能なツールが広く使われています。

このノートブックでは、Hugging Face `transformers` ライブラリを使って、いくつかの有名な事前学習済みモデルに付属するTokenizerを実際に動かし、その動作を確認します。
これにより、サブワード分割が実際のモデルでどのように機能しているかの具体的なイメージを掴むことを目的とします。

**このノートブックで学ぶこと:**
1.  Hugging Face `transformers` ライブラリの基本的なTokenizerの使い方。
2.  異なる事前学習済みモデル（例: BERT, GPT-2, T5）が採用しているサブワード分割戦略の比較。
3.  トークナイズ結果（トークン、ID、アテンションマスクなど）の確認。
4.  特殊トークン（`[CLS]`, `[SEP]`, `<|endoftext|>`など）の役割。

**前提知識:**
*   サブワード分割（BPE, WordPiece, SentencePiece）の基本的な概念の理解（NLP基礎(4)のノートブック）。
*   Pythonの基本的な操作。
*   (推奨) Hugging Face Transformersライブラリの概要についての知識（なくても進められます）。

**準備:**
Hugging Face `transformers` ライブラリと、それが依存する `tokenizers` ライブラリが必要です。
まだインストールしていない場合は、以下のコマンドでインストールしてください。
`pip install transformers tokenizers sentencepiece`

In [2]:
import torch # Hugging FaceのTokenizerはPyTorchテンソルも扱える
from transformers import AutoTokenizer, BertTokenizer, GPT2Tokenizer, T5Tokenizer

print(f"PyTorch Version: {torch.__version__}")
print("Hugging Face Transformers and Tokenizers are expected to be installed.")


PyTorch Version: 2.5.0+cu124
Hugging Face Transformers and Tokenizers are expected to be installed.


## 2. 様々な事前学習済みモデルのTokenizerを試す

Hugging Face `transformers` ライブラリの `AutoTokenizer` を使うと、モデル名を指定するだけで適切なTokenizerを自動的にロードできます。

### 2.1 BERT Tokenizer (WordPieceベース)

BERT (Bidirectional Encoder Representations from Transformers) は、WordPieceというサブワード分割アルゴリズムを使用しています。
WordPieceは、単語をより頻繁に出現する部分文字列に分割し、単語の先頭でないサブワードには通常 `##` というプレフィックスを付けます。

In [3]:
print("--- BERT Tokenizer (WordPiece) ---")
# 一般的なBERTの事前学習済みモデル名を指定
bert_model_name = 'bert-base-uncased' # 小文字化するBERTモデル

bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
print(f"'{bert_model_name}' のTokenizerをロードしました。")

sample_text_bert = "This is an example of using the BERT tokenizer with subword units, like 'tokenization'."

# テキストをトークンに分割
bert_tokens = bert_tokenizer.tokenize(sample_text_bert)
print(f"\nSample text: '{sample_text_bert}'")
print(f"BERT Tokens: {bert_tokens}")

# テキストをトークンIDに変換 (エンコード)
# add_special_tokens=True にすると、[CLS]や[SEP]が付加される
bert_encoded_input = bert_tokenizer.encode_plus(
    sample_text_bert, 
    add_special_tokens=True, 
    return_attention_mask=True, # アテンションマスクも取得
    return_tensors='pt' # PyTorchテンソルで返す
)
bert_input_ids = bert_encoded_input['input_ids']
bert_attention_mask = bert_encoded_input['attention_mask']
    
print("\nBERT Encoded Input IDs:\n", bert_input_ids)
print("Corresponding Tokens (manual check):")
print(bert_tokenizer.convert_ids_to_tokens(bert_input_ids.squeeze().tolist()))
print("Attention Mask:\n", bert_attention_mask) # パディングがないので全て1

# IDシーケンスを元のテキストに戻す (デコード)
# skip_special_tokens=True にすると、[CLS]や[SEP]が除去される
decoded_text_bert = bert_tokenizer.decode(bert_input_ids.squeeze(), skip_special_tokens=True)
print(f"\nDecoded text (from IDs, skipping special tokens): '{decoded_text_bert}'")
    
# 未知語の扱い
# BERTの語彙にない単語は [UNK] トークンになる
unknown_text_bert = "A very newwwwwoooord."
unknown_tokens_bert = bert_tokenizer.tokenize(unknown_text_bert)
print(f"\nText with unknown word: '{unknown_text_bert}'")
print(f"BERT Tokens: {unknown_tokens_bert}") # 'newwwwwoooord' が [UNK] になるはず

--- BERT Tokenizer (WordPiece) ---


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\syuuy\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\syuuy\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

'bert-base-uncased' のTokenizerをロードしました。

Sample text: 'This is an example of using the BERT tokenizer with subword units, like 'tokenization'.'
BERT Tokens: ['this', 'is', 'an', 'example', 'of', 'using', 'the', 'bert', 'token', '##izer', 'with', 'sub', '##word', 'units', ',', 'like', "'", 'token', '##ization', "'", '.']

BERT Encoded Input IDs:
 tensor([[  101,  2023,  2003,  2019,  2742,  1997,  2478,  1996, 14324, 19204,
         17629,  2007,  4942, 18351,  3197,  1010,  2066,  1005, 19204,  3989,
          1005,  1012,   102]])
Corresponding Tokens (manual check):
['[CLS]', 'this', 'is', 'an', 'example', 'of', 'using', 'the', 'bert', 'token', '##izer', 'with', 'sub', '##word', 'units', ',', 'like', "'", 'token', '##ization', "'", '.', '[SEP]']
Attention Mask:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

Decoded text (from IDs, skipping special tokens): 'this is an example of using the bert tokenizer with subword units, like ' tokenization '.'

T

**BERT Tokenizerのポイント:**
*   `bert-base-uncased` は、入力テキストを小文字化してからトークナイズします。
*   単語が語彙にない場合や、頻度の低いサブシーケンスは、より細かいサブワード（最終的には文字単位や `[UNK]` トークン）に分割されます。
*   `encode_plus` メソッドは、トークンIDだけでなく、アテンションマスクやトークンタイプID（NSP用）など、モデルへの入力に必要な情報をまとめて生成できます。
*   特殊トークン `[CLS]` はシーケンスの開始を表し、通常、文全体の表現ベクトルとして利用されます。`[SEP]` は文の区切りを示します。

### 2.2 GPT-2 Tokenizer (BPEベース)

GPT-2 (Generative Pre-trained Transformer 2) は、Byte Pair Encoding (BPE) に基づくサブワード分割を使用しています。
BPEは、最も頻繁に出現するバイト（文字）ペアを繰り返しマージしていくことで語彙を構築します。
GPT-2のTokenizerは、単語の先頭を示すために特別な記号（例: `Ġ`、スペースのメタキャラクタ）を使うことがあります。

In [5]:
print("\n--- GPT-2 Tokenizer (BPE) ---")
gpt2_model_name = 'gpt2' # 基本的なGPT-2モデル

gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_model_name)
print(f"'{gpt2_model_name}' のTokenizerをロードしました。")

# GPT-2は特殊なパディングトークンを持たないことがあるので、追加する
if gpt2_tokenizer.pad_token is None:
    gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    print("GPT-2 Tokenizerに [PAD] トークンを追加しました。")


sample_text_gpt2 = "This is an example for the GPT-2 tokenizer, which uses BPE tokenization."
    
gpt2_tokens = gpt2_tokenizer.tokenize(sample_text_gpt2)
print(f"\nSample text: '{sample_text_gpt2}'")
print(f"GPT-2 Tokens: {gpt2_tokens}")
# 'Ġ' が単語の開始（スペースを含む）を示していることに注目

# エンコード
gpt2_encoded_input = gpt2_tokenizer(
    sample_text_gpt2,
    add_special_tokens=True, # GPT-2は通常、文頭に<|endoftext|>などを自動で追加しない
    return_attention_mask=True,
    return_tensors='pt',
    padding=True, # バッチ処理を考慮してパディングを有効にする例
    truncation=True,
    max_length=30 # 例として最大長を設定
)
gpt2_input_ids = gpt2_encoded_input['input_ids']
gpt2_attention_mask = gpt2_encoded_input['attention_mask']

print("\nGPT-2 Encoded Input IDs (padded/truncated):\n", gpt2_input_ids)
print("Corresponding Tokens:")
print(gpt2_tokenizer.convert_ids_to_tokens(gpt2_input_ids.squeeze().tolist()))
print("Attention Mask:\n", gpt2_attention_mask)
    
# デコード
# GPT-2の出力は通常、<|endoftext|>のようなEOSトークンで終わる
decoded_text_gpt2 = gpt2_tokenizer.decode(gpt2_input_ids.squeeze(), skip_special_tokens=False) # 特殊トークンも表示
print(f"\nDecoded text (from IDs, with special tokens if any): '{decoded_text_gpt2}'")
    
decoded_text_gpt2_skip = gpt2_tokenizer.decode(gpt2_input_ids.squeeze(), skip_special_tokens=True) # 特殊トークンを除去
print(f"Decoded text (from IDs, skipping special tokens): '{decoded_text_gpt2_skip}'")


--- GPT-2 Tokenizer (BPE) ---
'gpt2' のTokenizerをロードしました。
GPT-2 Tokenizerに [PAD] トークンを追加しました。

Sample text: 'This is an example for the GPT-2 tokenizer, which uses BPE tokenization.'
GPT-2 Tokens: ['This', 'Ġis', 'Ġan', 'Ġexample', 'Ġfor', 'Ġthe', 'ĠG', 'PT', '-', '2', 'Ġtoken', 'izer', ',', 'Ġwhich', 'Ġuses', 'ĠB', 'PE', 'Ġtoken', 'ization', '.']

GPT-2 Encoded Input IDs (padded/truncated):
 tensor([[ 1212,   318,   281,  1672,   329,   262,   402, 11571,    12,    17,
         11241,  7509,    11,   543,  3544,   347, 11401, 11241,  1634,    13]])
Corresponding Tokens:
['This', 'Ġis', 'Ġan', 'Ġexample', 'Ġfor', 'Ġthe', 'ĠG', 'PT', '-', '2', 'Ġtoken', 'izer', ',', 'Ġwhich', 'Ġuses', 'ĠB', 'PE', 'Ġtoken', 'ization', '.']
Attention Mask:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

Decoded text (from IDs, with special tokens if any): 'This is an example for the GPT-2 tokenizer, which uses BPE tokenization.'
Decoded text (from IDs, skipping special tokens): 'T

**GPT-2 Tokenizerのポイント:**
*   GPT-2のBPEは、文字レベルから始まり、頻出するバイトペアをマージしていきます。
*   トークン化された結果では、単語の区切りや単語内での分割が、BERTのWordPieceとは異なる形で見られます（例: `Ġ` プレフィックス）。
*   GPT-2は主にテキスト生成に使われるため、デコード時には生成されたIDシーケンスを人間が読めるテキストに戻すことが重要になります。
*   事前学習済みモデルによっては、パディングトークンが明示的に設定されていない場合があるため、バッチ処理などを行う際には手動で追加設定することがあります。

### 2.3 T5 Tokenizer (SentencePieceベース)

T5 (Text-to-Text Transfer Transformer) は、SentencePieceというサブワード分割アルゴリズムを使用しています。
SentencePieceは、言語非依存であり、生のテキストから直接サブワードモデルを学習できる特徴があります。空白文字も通常の文字と同様に扱い、メタシンボル（例: ` ` (U+2581)）で表現します。

In [6]:
print("\n--- T5 Tokenizer (SentencePiece) ---")
t5_model_name = 't5-small' # T5の小さなバリアント

t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
print(f"'{t5_model_name}' のTokenizerをロードしました。")

sample_text_t5 = "SentencePiece makes subword tokenization language-independent. こんにちは。"
    
t5_tokens = t5_tokenizer.tokenize(sample_text_t5)
print(f"\nSample text: '{sample_text_t5}'")
print(f"T5 Tokens (SentencePiece): {t5_tokens}")
# ' ' (U+2581) が単語の開始（スペースを含む）を示していることに注目
# 日本語もサブワードに分割される

# エンコード (T5はタスクごとにプレフィックスを付けることが多いが、ここでは単純なエンコード)
t5_encoded_input = t5_tokenizer(
    sample_text_t5,
    return_tensors='pt',
    padding=True,
    truncation=True,
    max_length=30
)
t5_input_ids = t5_encoded_input['input_ids']
t5_attention_mask = t5_encoded_input['attention_mask']

print("\nT5 Encoded Input IDs (padded/truncated):\n", t5_input_ids)
print("Corresponding Tokens:")
print(t5_tokenizer.convert_ids_to_tokens(t5_input_ids.squeeze().tolist()))
print("Attention Mask:\n", t5_attention_mask)
    
# デコード
# T5は通常、シーケンスの終わりにEOSトークン (</s>) を付加する
decoded_text_t5 = t5_tokenizer.decode(t5_input_ids.squeeze(), skip_special_tokens=False)
print(f"\nDecoded text (from IDs, with special tokens): '{decoded_text_t5}'")
    
decoded_text_t5_skip = t5_tokenizer.decode(t5_input_ids.squeeze(), skip_special_tokens=True)
print(f"Decoded text (from IDs, skipping special tokens): '{decoded_text_t5_skip}'")


--- T5 Tokenizer (SentencePiece) ---


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

c:\Users\syuuy\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\syuuy\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

't5-small' のTokenizerをロードしました。

Sample text: 'SentencePiece makes subword tokenization language-independent. こんにちは。'
T5 Tokens (SentencePiece): ['▁Sen', 't', 'ence', 'P', 'i', 'e', 'ce', '▁makes', '▁sub', 'word', '▁token', 'ization', '▁language', '-', 'in', 'dependent', '.', '▁', 'こんにちは。']

T5 Encoded Input IDs (padded/truncated):
 tensor([[ 4892,    17,  1433,   345,    23,    15,   565,   656,   769,  6051,
         14145,  1707,  1612,    18,    77, 17631,     5,     3,     2,     1]])
Corresponding Tokens:
['▁Sen', 't', 'ence', 'P', 'i', 'e', 'ce', '▁makes', '▁sub', 'word', '▁token', 'ization', '▁language', '-', 'in', 'dependent', '.', '▁', '<unk>', '</s>']
Attention Mask:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

Decoded text (from IDs, with special tokens): 'SentencePiece makes subword tokenization language-independent. <unk></s>'
Decoded text (from IDs, skipping special tokens): 'SentencePiece makes subword tokenization language-independent. '


**T5 Tokenizer (SentencePiece) のポイント:**
*   SentencePieceは、生のテキストから直接学習するため、言語固有の前処理（単語分割など）が不要です。
*   空白文字を特別なメタシンボル ` ` (U+2581) として扱うことで、トークナイズ結果から元の文を完全に復元できます（Lossless Tokenization）。
*   BPEアルゴリズムまたはUnigram Language Modelアルゴリズムに基づいてサブワード語彙を構築できます（T5はUnigram LMベースが多いです）。
*   多言語対応にも優れています。

## 3. トークナイザの共通パラメータと出力

多くのHugging Face Tokenizerは、テキストをエンコードする際に共通の引数を取ります。

*   `text` または `text_pair`: トークナイズする単一のテキストまたはテキストペア。
*   `add_special_tokens=True`: `[CLS]`, `[SEP]`, `<bos>`, `<eos>` のようなモデル固有の特殊トークンを自動的に付加するかどうか。
*   `padding`:
    *   `False` (デフォルト): パディングしない。
    *   `True` または `'longest'`: バッチ内で最も長いシーケンスに合わせてパディング。
    *   `'max_length'`: `max_length` 引数で指定された長さに合わせてパディング（または切り詰め）。
*   `truncation=True`: `max_length` を超える場合にシーケンスを切り詰める。
*   `max_length`: パディングまたは切り詰めの最大長。
*   `return_tensors`: 返り値の型を指定 (`'pt'` for PyTorch, `'tf'` for TensorFlow, `'np'` for NumPy)。
*   `return_attention_mask=True`: アテンションメカニズムでパディングトークンを無視するためのマスクを返す。
*   `return_token_type_ids=True`: (主にBERTなど) 2つのシーケンスを入力とする場合に、各トークンがどちらのシーケンスに属するかを示すIDを返す。

**主な返り値（辞書形式）:**
*   `input_ids`: トークンIDのシーケンス。
*   `attention_mask`: アテンションマスク（1が通常のトークン、0がパディングトークン）。
*   `token_type_ids`: トークンタイプID（必要な場合）。

In [7]:
print("\n--- Tokenizerの共通パラメータと出力の確認 (BERT例) ---")
bert_tokenizer_example = AutoTokenizer.from_pretrained('bert-base-uncased')
text_pair_example = ("First sentence.", "Second sentence, much longer.")

encoded_pair = bert_tokenizer_example(
    text_pair_example[0], text_pair_example[1], # テキストペアを渡す
    add_special_tokens=True,
    padding='max_length',       # 最大長に合わせてパディング
    truncation=True,            # 最大長を超える場合は切り詰め
    max_length=20,              # 例として最大長20
    return_tensors='pt',
    return_attention_mask=True,
    return_token_type_ids=True  # NSPタスクなどで使用
)

print("Input Text 1:", text_pair_example[0])
print("Input Text 2:", text_pair_example[1])
print("\nEncoded Input IDs:\n", encoded_pair['input_ids'])
print("Decoded Tokens:\n", bert_tokenizer_example.convert_ids_to_tokens(encoded_pair['input_ids'].squeeze().tolist()))
print("Attention Mask:\n", encoded_pair['attention_mask'])
print("Token Type IDs:\n", encoded_pair['token_type_ids'])
# Token Type IDs: 最初の文 ([CLS]含む) が0、2番目の文 ([SEP]含む) が1 になる


--- Tokenizerの共通パラメータと出力の確認 (BERT例) ---
Input Text 1: First sentence.
Input Text 2: Second sentence, much longer.

Encoded Input IDs:
 tensor([[ 101, 2034, 6251, 1012,  102, 2117, 6251, 1010, 2172, 2936, 1012,  102,
            0,    0,    0,    0,    0,    0,    0,    0]])
Decoded Tokens:
 ['[CLS]', 'first', 'sentence', '.', '[SEP]', 'second', 'sentence', ',', 'much', 'longer', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Attention Mask:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])
Token Type IDs:
 tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])


## 4. 考察とまとめ

このノートブックでは、Hugging Face `transformers` ライブラリを使って、BERT (WordPiece)、GPT-2 (BPE)、T5 (SentencePiece) といった代表的な事前学習済みモデルのTokenizerを簡単に利用し、その動作を確認しました。

*   **サブワード分割の多様性:** モデルごとに採用しているサブワード分割アルゴリズムや、特殊トークンの扱い、単語の内部構造の表現方法（例: BERTの`##`、GPT-2の`Ġ`、SentencePieceの` `）が異なることがわかります。
*   **ライブラリの利便性:** `AutoTokenizer` を使うことで、モデル名を指定するだけで適切なTokenizerをロードでき、エンコード・デコード処理も統一的なインターフェースで行えます。パディング、切り詰め、特殊トークンの付加といった煩雑な処理も簡単に行えます。
*   **モデルへの入力形式:** Tokenizerは、テキストをモデルが直接処理できる数値形式（トークンID、アテンションマスクなど）に変換する重要な役割を担っています。

サブワード分割は、現代のNLPモデルの性能と効率を支える基盤技術です。これらのTokenizerがどのように動作するかを理解することは、モデルの挙動を把握し、適切に利用する上で非常に重要です。

これで、自然言語処理の基礎的な前処理から表現方法、そしてサブワード分割までの学習セクションは一区切りとなります。